In [2]:
import pandas as pd
import numpy as np
import psycopg2
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option("display.max_colwidth", 200)

In [11]:

conn = psycopg2.connect(host="localhost",database="mimic", user="postgres", password='postgres', port = '5432')
cursor = conn.cursor()

In [4]:
sql = """
SELECT row_id, icd9_code, short_title, long_title
  FROM mimiciii.d_icd_diagnoses
  where long_title like '%kidney disease%'
  order by icd9_code
  """
df = pd.read_sql_query(sql, conn)

In [5]:
# diagnosis that correspond to kidney diease
# currently pulling out 585.6 and 585.6 for end state renal disease which requires dialysis.
# should we pull out others? 
df

,row_id,icd9_code,short_title,long_title
0,3142,28521,Anemia in chr kidney dis,Anemia in chronic kidney disease
1,4311,40300,Mal hy kid w cr kid I-IV,"Hypertensive chronic kidney disease, malignant, with chronic kidney disease stage I through stage IV, or unspecified"
2,4312,40301,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant, with chronic kidney disease stage V or end stage renal disease"
3,4313,40310,Ben hy kid w cr kid I-IV,"Hypertensive chronic kidney disease, benign, with chronic kidney disease stage I through stage IV, or unspecified"
4,4314,40311,Ben hyp kid w cr kid V,"Hypertensive chronic kidney disease, benign, with chronic kidney disease stage V or end stage renal disease"
5,4315,40390,Hy kid NOS w cr kid I-IV,"Hypertensive chronic kidney disease, unspecified, with chronic kidney disease stage I through stage IV, or unspecified"
6,4316,40391,Hyp kid NOS w cr kid V,"Hypertensive chronic kidney disease, unspecified, with chronic kidney disease stage V or end stage renal disease"
7,4317,40400,Mal hy ht/kd I-IV w/o hf,"Hypertensive heart and chronic kidney disease, malignant, without heart failure and with chronic kidney disease stage I through stage IV, or unspecified"
8,4318,40401,Mal hyp ht/kd I-IV w hf,"Hypertensive heart and chronic kidney disease, malignant, with heart failure and with chronic kidney disease stage I through stage IV, or unspecified"
9,4319,40402,Mal hy ht/kd st V w/o hf,"Hypertensive heart and chronic kidney disease, malignant, without heart failure and with chronic kidney disease stage V or end stage renal disease"


In [17]:
# of patients with chronic dialysis
sqlfile = open('C:/Users/310050083/Documents/echo/src/chronic_dialysis.sql', 'r')
#cursor.execute(sqlfile.read(), conn)
#df = pd.read_sql_query(sqlfile.read(), conn)
sql = """
SELECT subject_id, hadm_id, seq_num, icd9_code
  FROM mimiciii.diagnoses_icd
  where icd9_code in (
    '5855' -- Chron kidney dis stage V
    ,'5856' -- End stage renal disease
    , '40301' -- "Mal hyp kid w cr kid V"
    , '40311' -- "Ben hyp kid w cr kid V"
    , '40391' -- "Hyp kid NOS w cr kid V"
    , '40402' -- Mal hy ht/kd st V w/o hf"
    , '40403' -- Mal hyp ht/kd stg V w hf"
    , '40412' -- Ben hy ht/kd st V w/o hf"
    , '40413' -- Ben hyp ht/kd stg V w hf"
    , '40492' -- Hy ht/kd NOS st V w/o hf"
    , '40493' -- Hyp ht/kd NOS st V w hf
    )
  order by subject_id
"""
df = pd.read_sql_query(sql, conn)


In [23]:
print "Number of patients with chronic dialysis"
len(df.subject_id.unique())

Number of patients with chronic dialysis


1928

In [14]:
# getting codes for BNP
sql = """
SELECT row_id, itemid, label, abbreviation, dbsource, linksto, category, 
       unitname, param_type, conceptid
  FROM mimiciii.d_items
  where label like '%BNP%' or label like '%Troponin%'
  order by label;
"""
df = pd.read_sql_query(sql, conn)

In [15]:
df.head()

,row_id,itemid,label,abbreviation,dbsource,linksto,category,unitname,param_type,conceptid
0,4600,7294,BNP,None,carevue,chartevents,None,None,None,None
1,14760,227446,Brain Natiuretic Peptide (BNP),Brain Natiuretic Peptide (BNP),metavision,chartevents,Labs,None,Numeric with tag,None
2,1139,851,Troponin,None,carevue,chartevents,Enzymes,None,None,None
3,14751,227429,Troponin-T,Troponin-T,metavision,chartevents,Labs,None,Numeric with tag,None
4,14079,225622,ZBrain Natiuretic Peptide (BNP),ZBrain Natiuretic Peptide (BNP),metavision,chartevents,Labs,None,Numeric,None


In [24]:
# example of patient with echo who had many fluid entries

sql = """
SELECT *
  FROM mimiciii.inputevents_mv 
  where itemid in (
	 '225158' -- NaCl 0.9%
	,'220949' -- Dextrose 5%
	,'225828' -- LR
	,'225823' -- D5 1/2NS
	,'220862' -- Albumin 25%
	,'220970' -- Fresh Frozen Plasma
	,'225159' -- NaCl 0.45%
	,'220864' -- Albumin 5%
	,'226364' -- OR Crystalloid Intake
	,'220995' -- Sodium Bicarbonate 8.4%
	,'225916' -- TPN w/ Lipids
	,'225825' -- D5NS
	,'225917' -- TPN without Lipids
	,'225161' -- NaCl 3% (Hypertonic Saline)
	,'225827' -- D5LR
	,'225941' -- D5 1/4NS
	,'226371' -- OR Cryoprecipitate Intake
  ) and icustay_id = 241249
  order by subject_id, starttime
  """
df = pd.read_sql_query(sql, conn)


In [26]:
pd.set_option("display.max_rows",101)
df

,row_id,subject_id,hadm_id,icustay_id,starttime,endtime,itemid,amount,amountuom,rate,...,totalamountuom,isopenbag,continueinnextdept,cancelreason,statusdescription,comments_editedby,comments_canceledby,comments_date,originalamount,originalrate
0,1536944,36,165660,241249,2134-05-12 07:07:00,2134-05-12 07:48:00,225158,6.833333,ml,10.000000,...,ml,0,0,0,Stopped,None,None,None,1000.000000,10.000000
1,1536895,36,165660,241249,2134-05-12 07:45:00,2134-05-12 13:59:00,225823,779.166654,ml,124.999998,...,ml,0,0,0,Changed,None,None,None,1000.000000,125.000000
2,1536794,36,165660,241249,2134-05-12 11:00:00,2134-05-13 00:03:00,220949,65.250002,ml,5.000000,...,ml,0,0,0,Changed,None,None,None,250.000000,5.000000
3,1536836,36,165660,241249,2134-05-12 13:59:00,2134-05-13 07:45:00,225823,177.666670,ml,10.000000,...,ml,0,0,0,Stopped,None,None,None,220.833330,10.000000
4,1536958,36,165660,241249,2134-05-12 14:00:00,2134-05-12 14:01:00,220949,100.000000,ml,NaN,...,ml,0,0,0,FinishedRunning,None,None,None,100.000000,0.000000
5,1536894,36,165660,241249,2134-05-12 16:00:00,2134-05-12 22:30:00,225158,13.000001,ml,2.000000,...,ml,0,0,0,Changed,None,None,None,100.000000,2.000000
6,1536965,36,165660,241249,2134-05-12 22:30:00,2134-05-13 05:03:00,225158,19.650002,ml,3.000000,...,ml,0,0,0,Changed,None,None,None,87.000000,3.000000
7,1536889,36,165660,241249,2134-05-12 23:30:00,2134-05-12 23:31:00,225158,500.000000,ml,NaN,...,ml,0,0,0,FinishedRunning,None,None,None,500.000000,500.000000
8,1536914,36,165660,241249,2134-05-13 00:03:00,2134-05-13 06:01:00,220949,53.685472,ml,8.997565,...,ml,0,0,0,Changed,None,None,None,184.750000,9.000000
9,1536854,36,165660,241249,2134-05-13 01:39:00,2134-05-13 01:40:00,225158,500.000000,ml,NaN,...,ml,0,0,0,FinishedRunning,None,None,None,500.000000,500.000000
